# Tap dependent impedance of transformers

With an adjustment of a tap position of a transformer, the number of windings is adjusted, which influences the impedence. The user can provide **trafo_characteristic_table** of the dependance of the variables **voltage ratio**, **angle in degree**, **vk_percent** and **vkr_percent** on the tap position. A helper function in pandapower control module converts the provided points in a characteristic object with quadratic spline interpolation (or a user-defined characteristic object with a custom interpolation approach) and writes it in the **trafo_characteristic_spline** table. Finally, a diagnostic function checks for inconsistensies and provides information to the user.

In [ ]:
from pandapower.create import *
from pandapower.run import runpp
from pandapower.control import SplineCharacteristic, plot_characteristic, TapDependentImpedance, create_trafo_characteristic_object, trafo_characteristic_table_diagnostic
from pandapower.file_io import from_json_string, to_json

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
def add_trafo_connection(net, hv_bus, element_type="trafo"):
    cb = create_bus(net, vn_kv=0.4)
    create_load(net, cb, 0.2, 0.05)
    
    if element_type=="trafo3w":
        cbm = create_bus(net, vn_kv=0.9)
        create_load(net, cbm, 0.1, 0.03)
        create_transformer3w_from_parameters(net, hv_bus=hv_bus, mv_bus=cbm, lv_bus=cb,
                                                vn_hv_kv=20., vn_mv_kv=0.9, vn_lv_kv=0.45, sn_hv_mva=0.6, 
                                                sn_mv_mva=0.5, sn_lv_mva=0.4, vk_hv_percent=1., 
                                                vk_mv_percent=1., vk_lv_percent=1., vkr_hv_percent=0.3, 
                                                vkr_mv_percent=0.3, vkr_lv_percent=0.3, pfe_kw=0.2, 
                                                i0_percent=0.3, tap_neutral=0., tap_pos=2, tap_changer_type="Ratio",
                                                tap_step_percent=1., tap_min=-2, tap_max=2)
    else:
        create_transformer(net, hv_bus=hv_bus, lv_bus=cb, std_type="0.25 MVA 20/0.4 kV", tap_pos=2)

def create_net():
    net = create_empty_network()
    vn_kv = 20
    b1 = create_bus(net, vn_kv=vn_kv)
    create_ext_grid(net, b1, vm_pu=1.01)
    b2 = create_bus(net, vn_kv=vn_kv)
    l1 = create_line_from_parameters(net, b1, b2, 12.2, r_ohm_per_km=0.08, x_ohm_per_km=0.12,
                                        c_nf_per_km=300, max_i_ka=.2, df=.8)
    for i in range(2):
        add_trafo_connection(net, b2)
        net.trafo.loc[i, "id_characteristic_table"] = i
        net.trafo.loc[i,"tap_dependency_table"] = True

    # Adding the characteristics data for two- and three-winding transformers into the trafo_characteristic_table
    net['trafo_characteristic_table'] = pd.DataFrame(
    columns=['id_characteristic', 'step', 'voltage_ratio', 'angle_deg', 'vk_percent',
             'vkr_percent', 'vkr_hv_percent', 'vkr_mv_percent',
             'vkr_lv_percent', 'vk_hv_percent', 'vk_mv_percent',
             'vk_lv_percent'])
    trafo_data = {
        'id_characteristic': [0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
        'step': [-2, -1, 0, 1, 2, -2, -1, 0, 1, 2],
        'voltage_ratio': [0.95, 0.97, 1.0, 1.03, 1.05, 0.95, 0.97, 1.0, 1.03, 1.05],
        'angle_deg': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        'vk_percent': [5, 5.2, 6, 6.8, 7, 5, 5.2, 6, 6.8, 7],
        'vkr_percent': [1.3, 1.4, 1.44, 1.5, 1.6, 1.3, 1.4, 1.44, 1.5, 1.6],
        'vkr_hv_percent': np.nan,
        'vkr_mv_percent': np.nan,
        'vkr_lv_percent': np.nan,
        'vk_hv_percent': np.nan,
        'vk_mv_percent': np.nan,
        'vk_lv_percent': np.nan
    }

    net['trafo_characteristic_table'] = pd.DataFrame(trafo_data)

    trafo3w_data = {
        'id_characteristic': [2, 2, 2, 2, 2, 3, 3, 3, 3, 3],
        'step': [-2, -1, 0, 1, 2, -2, -1, 0, 1, 2],
        'voltage_ratio': [0.95, 0.97, 1.0, 1.03, 1.05, 0.95, 0.97, 1.0, 1.03, 1.05],
        'angle_deg': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        'vk_percent': np.nan,
        'vkr_percent': np.nan,
        'vkr_hv_percent': [0.27, 0.28, 0.3, 0.32, 0.33, 0.27, 0.28, 0.3, 0.32, 0.33],
        'vkr_mv_percent': [0.27, 0.28, 0.3, 0.32, 0.33, 0.27, 0.28, 0.3, 0.32, 0.33],
        'vkr_lv_percent': [0.27, 0.28, 0.3, 0.32, 0.33, 0.27, 0.28, 0.3, 0.32, 0.33],
        'vk_hv_percent': [0.85, 0.9, 1, 1.1, 1.15, 0.85, 0.9, 1, 1.1, 1.15],
        'vk_mv_percent': [0.85, 0.9, 1, 1.1, 1.15, 0.85, 0.9, 1, 1.1, 1.15],
        'vk_lv_percent': [0.85, 0.9, 1, 1.1, 1.15, 0.85, 0.9, 1, 1.1, 1.15],
    }
    data_frame = pd.DataFrame(trafo3w_data)
    net['trafo_characteristic_table'] = pd.concat([net['trafo_characteristic_table'], data_frame], sort=False)
    return net

In [ ]:
net = create_net()

We can create the characteristics by generating characteristic objects and assigning them to the respective columns in the **trafo_characteristic_spline table**. To create these characteristics, a convenience function can be used to initialize and configure the characteristic objects for **voltage ratio**, **angle in degree**, **vk_percent** and **vkr_percent**. This function creates the characteristics for all transfomers within the specified network. It is essential that the input parameters are provided in the correct order to ensure proper functionality.

In [ ]:
create_trafo_characteristic_object(net)
net.trafo.loc[1,"id_characteristic_table"] = np.nan
net.trafo.loc[1,"tap_dependency_table"] = False
net.trafo

The characteristic set in the convenience function is the SplineCharacteristic object that relies on quadratic spline interpolation with interp1d from SciPy. Let us check the characteristic:

In [ ]:
plot_characteristic(net["trafo_characteristic_spline"]["vk_percent_characteristic"].loc[0], -2, 2,
                               xlabel='Tap position "tap_pos"', ylabel='Value of "vk_percent"')
plot_characteristic(net["trafo_characteristic_spline"]["vkr_percent_characteristic"].loc[0], -2, 2,
                               xlabel='Tap position "tap_pos"', ylabel='Value of "vkr_percent"')
runpp(net)

Let us verify that the adjusted values were considered in the calculation:

In [ ]:
net.res_bus.loc[[2,3]]

We can see that the voltage values are different, even though the connections are identical. Now let us compare the computation time:

In [ ]:
net2 = create_net()
net2.trafo["id_characteristic_table"] = np.nan
net2.trafo["tap_dependency_table"] = False

In [ ]:
%timeit runpp(net2)

In [ ]:
%timeit runpp(net)

The calculation with tap dependent impedance is somewhat slower.

Now, let us compare the performance of the controller for reference

In [ ]:
net3 = create_net()
net3.trafo.loc[1,"id_characteristic_table"] = np.nan
net3.trafo.loc[1,"tap_dependency_table"] = False
SplineCharacteristic(net3, [-2, -1, 0, 1, 2], [5, 5.2, 6, 6.8, 7])
SplineCharacteristic(net3, [-2, -1, 0, 1, 2], [1.3, 1.4, 1.44, 1.5, 1.6])
TapDependentImpedance(net3, [0], 0, output_variable="vk_percent")
TapDependentImpedance(net3, [0], 1, output_variable="vkr_percent")

In [ ]:
%timeit runpp(net3, run_control=True)

The computational time is substantially higher!

The results from using the characteristics in pandapower directly and via controllers are identical:

In [ ]:
net3.res_bus.loc[[2,3]]

In [ ]:
assert np.allclose(net.res_bus.vm_pu, net3.res_bus.vm_pu, atol=1e-6, rtol=0)
assert np.allclose(net.res_bus.va_degree, net3.res_bus.va_degree, atol=1e-6, rtol=0)

# 3-Winding Transformers

In [ ]:
net4 = create_net()
net5 = create_net()
for i in range(2):
    add_trafo_connection(net4, net4.trafo.at[0, 'hv_bus'], "trafo3w")
    add_trafo_connection(net5, net5.trafo.at[0, 'hv_bus'], "trafo3w")

In [ ]:
net4.trafo3w.loc[0,"id_characteristic_table"] = 2
net4.trafo3w.loc[0,"tap_dependency_table"] = True
create_trafo_characteristic_object(net4)

In [ ]:
SplineCharacteristic(net5, [-2, -1, 0, 1, 2], [0.85, 0.9, 1, 1.1, 1.15])
SplineCharacteristic(net5, [-2, -1, 0, 1, 2], [0.27, 0.28, 0.3, 0.32, 0.33])

In [ ]:
TapDependentImpedance(net5, [0], 0, output_variable="vk_hv_percent",  element="trafo3w")
TapDependentImpedance(net5, [0], 1, output_variable="vkr_hv_percent", element="trafo3w")
TapDependentImpedance(net5, [0], 0, output_variable="vk_mv_percent",  element="trafo3w")
TapDependentImpedance(net5, [0], 1, output_variable="vkr_mv_percent", element="trafo3w")
TapDependentImpedance(net5, [0], 0, output_variable="vk_lv_percent",  element="trafo3w")
TapDependentImpedance(net5, [0], 1, output_variable="vkr_lv_percent", element="trafo3w")

In [ ]:
%timeit runpp(net4)

In [ ]:
%timeit runpp(net5, run_control=True)

In [ ]:
assert np.allclose(net4.res_bus.vm_pu, net5.res_bus.vm_pu, atol=1e-6, rtol=0)
assert np.allclose(net4.res_bus.va_degree, net5.res_bus.va_degree, atol=1e-6, rtol=0)
pd.merge(net4.res_bus[["vm_pu", "va_degree"]], net5.res_bus[["vm_pu", "va_degree"]], 
         left_index=True, right_index=True)

Also in the case of the 3-winding transformers, the results for direct calculation in pandapower and in controllers match.

# Undefined characteristics

If some transformers have characteristics that do not define all columns, the missing characteristics are ignored by default. However, if **tap_dependency_table** is set to True, one or more characteristics are missing, a warning is issued, followed by an error during the load flow calculation.

In [ ]:
net6 = create_net()

In [ ]:

net6.trafo.loc[1,"tap_dependency_table"] = False
net6["trafo_characteristic_table"].loc[net["trafo_characteristic_table"]["id_characteristic"]==1, "vk_percent"] = np.nan
create_trafo_characteristic_object(net6)
net6.trafo

In [ ]:
%timeit runpp(net6)

In [ ]:
#this would raise a warning:
#net6.trafo.loc[1,"tap_dependency_table"] = True
#trafo_characteristic_table_diagnostic(net6)

In [ ]:
#this would raise an error:
#runpp(net6)

# Creating transformers with characteristics

The functions to create transformers have been updated to include the parameters **tap_dependency_table** and **id_characteristic_table** in trasformer which help to create the characteristics object and add the references to the **id_characteristic_spline** in trasformer using **create_trafo_characteristic_object** function.

In [ ]:
net_create = create_net()

In [ ]:
net_create.trafo

In [ ]:
create_transformer(net_create, hv_bus=net_create.trafo.at[0, 'hv_bus'],
                      lv_bus=net_create.trafo.at[0, 'lv_bus'], std_type="0.25 MVA 20/0.4 kV", tap_pos=2, 
                      tap_dependency_table=True, id_characteristic_table = 0)
create_trafo_characteristic_object(net_create)

In [ ]:
net_create.trafo

In [ ]:
runpp(net_create)

# File I/O

The file I/O for characteristics is now implemented via adding the characteristic table:

In [ ]:
net2 = from_json_string(to_json(net))
create_trafo_characteristic_object(net2) # need to recreate the object becuase it is binary

net2["trafo_characteristic_spline"]

De-serialization is working:

In [ ]:
plot_characteristic(net2["trafo_characteristic_spline"]["vk_percent_characteristic"].loc[0], -2, 2,
                               xlabel='Tap position "tap_pos"', ylabel='Value of "vk_percent"')

# Diagnostic

The diagnostic function for transformer characteristics checks the following:
 
* Are there transformer characteristic table and tap-dependent characteristics?
* Are any characteristics missing?
* Are there **tap_dependency_table**, **tap_changer_type** and **id_characteristic_table** missing in net.trafo or net.trafo3w?
* Check if all relevant columns for creating transformer characteristics are populated in the **trafo_characteristic_table**
* Check the data types of **tap_dependency_table** and **id_characteristic_table**
* Are there any missing **id_characteristic_table** values in the **trafo_characteristic_table**?

The results are displayed to the user with the help of warning statements.

In [ ]:
net.trafo

In [ ]:
trafo_characteristic_table_diagnostic(net)

In [ ]:
trafo_characteristic_table_diagnostic(net6)

In [ ]:
net_diagnostics = net
net_diagnostics.trafo.loc[0, "id_characteristic_table"] = None
trafo_characteristic_table_diagnostic(net_diagnostics)
#UserWarning: trafo: found 1 transformer(s) with not both tap_dependency_table and id_characteristic_table parameters populated. Power flow calculation will raise an error.
#  warnings.warn(f"{trafo_table}: found {mismatch_a} transformer(s) with not both "

In [ ]:
net_diagnostics.trafo.loc[0, "id_characteristic_table"] = 0
net_diagnostics.trafo.loc[0, "tap_changer_type"] = None
trafo_characteristic_table_diagnostic(net_diagnostics)
#UserWarning: trafo: found 1 transformer(s) with tap_dependency_table set to True and tap_changer_type parameter not populated. The characteristics from trafo_characteristic_table will not be considered.
 # warnings.warn(f"{trafo_table}: found {mismatch_b} transformer(s) with tap_dependency_table set to "